In [17]:
%reload_ext autoreload
%autoreload 2


In [18]:

import gc
import sys
sys.path.insert(0,"./src")

import methods
import glob
import numpy as np
import scipy
import matplotlib.pyplot as plt
# from math import hypot

X_paths = glob.glob("/Users/jackh/Documents/2023/Turbulence Research/Data/trajs/*.csv")

traj = methods.Trajectory(X=X_paths,t=range(len(X_paths)))

In [2]:
traj.n_t

# x = np.array(range(10))

# methods.dynamic_laplacian(traj)

x1 = traj.Xat(0)
# x2 = traj.Xat(0)

eps = methods.nndist(x1[:100,:],k=7)

kdtree = scipy.spatial.KDTree(x1)
x = kdtree.sparse_distance_matrix(kdtree,eps/np.sqrt(2)).tocsr()


# del x 
# gc.collect()

# x1 = np.array([[1,2],[2,3]])
# x2 = np.array([[1,2],[2,3]])

# scipy.spatial.distance_matrix(x1[:3,:],x2[:3,:])

# kdtree = scipy.spatial.KDTree(x1)
# x = kdtree.sparse_distance_matrix(kdtree,1)


In [152]:
x

<44991x44991 sparse matrix of type '<class 'numpy.float64'>'
	with 289907 stored elements in Compressed Sparse Row format>

In [71]:
del x 
gc.collect()

0

In [151]:
temp = x.data

x.data = methods.iso_kernel(temp,0,eps)

print(x.data)

# np.sum(x,axis=0)

[1.         0.82605705 0.82248711 ... 0.89921589 0.8470526  1.        ]


In [30]:
# sys.getsizeof(x)

x1 = traj.Xat(0)
kdtree = scipy.spatial.KDTree(x1)
nearest_dist, _ = kdtree.query(x1, k=2)


del kdtree
gc.collect()

print(nearest_dist)


[[0.         0.28812058]
 [0.         0.28601738]
 [0.         0.26934699]
 ...
 [0.         0.25514139]
 [0.         0.25123737]
 [0.         0.21482341]]


In [51]:
x1 = traj.Xat(0)
methods.nndist(x1[:100,:],k=5)

0.3760907123126933

In [162]:
methods.dynamic_laplacian_sl(x1)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/jackh/opt/anaconda3/envs/gordon_original/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/9v/f6c01sn97rb6v_y68jr4gcn80000gn/T/ipykernel_12511/1453643387.py", line 1, in <module>
    methods.dynamic_laplacian_sl(x1)
  File "/Users/jackh/Documents/2023/Turbulence Research/Code/./src/methods.py", line 213, in dynamic_laplacian_sl
    temp.data = iso_kernel(temp.data,0,eps)
                           ^^^^^^^^^
  File "/Users/jackh/opt/anaconda3/envs/gordon_original/lib/python3.11/site-packages/scipy/sparse/_base.py", line 771, in __getattr__
    raise AttributeError(attr + " not found")
AttributeError: data not found

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jackh/opt/anaconda3/envs/gordon_original/lib/python3.11/site-packages/IPython/core/interactivesh

In [125]:
methods.iso_kernel(x1[0:10,:],0,eps)

array([1.43762724e-108, 1.19075504e-107, 1.25158994e-109, 1.03207042e-107,
       5.55624143e-107, 4.82997299e-106, 3.94311020e-110, 9.27584147e-109,
       2.33624202e-107, 1.65410905e-106])

In [161]:
x.data

array([1.        , 0.82605705, 0.82248711, ..., 0.89921589, 0.8470526 ,
       1.        ])

In [16]:
x1 = traj.Xat(0)

X = x1

alpha = 1
eps = methods.nndist(X,k=7)
n_s = np.shape(X)[0] # X is a square matrix

kdtree = scipy.spatial.KDTree(X)
temp = kdtree.sparse_distance_matrix(kdtree,eps).tocoo()
temp.data = methods.iso_kernel(temp.data,0,eps)

# q_bar = np.power((np.sum(temp, axis=1)/n_s),alpha)
# temp.data = np.divide(temp.data,np.multiply(q_bar[temp.row], q_bar[temp.col]).T).A1
# d_barsum = np.sum(temp, axis=0)/n_s
# temp.data = np.divide(temp.data,d_barsum.A1[temp.col])

q_bar = np.power((np.sum(temp, axis=1)),alpha)
qq_bar = np.multiply(q_bar[temp.row], q_bar[temp.col]).A1
temp.data = np.divide(temp.data, qq_bar)
d_barsum = np.sum(temp, axis=1)
temp.data = np.divide(temp.data,d_barsum[temp.col].T).A1

np.sum(temp, axis=1)


matrix([[1.04198849],
        [1.07602043],
        [1.03573067],
        ...,
        [1.06959707],
        [0.99599889],
        [1.04407811]])

In [11]:
# q_bar
d_barsum.T
temp.data
np.divide(temp.data,d_barsum[temp.col].T).A1
# qq_bar



array([1.38610992, 1.27678584, 0.89719282, ..., 0.84650774, 0.93018355,
       0.99175021])

In [25]:
print(temp.row)
print(temp.col)
temp.data
# q_bar[temp.row]
# q_bar[temp.col].T
np.divide(temp.data,np.multiply(q_bar[temp.row], q_bar[temp.col]).T).T
# temp / (q_bar * q_bar.T)


[    0     1     3 ... 44988 44989 44990]
[    0     0     0 ... 44990 44990 44990]


matrix([[7.65979445e+15],
        [8.34948181e+15],
        [4.14630212e+15],
        ...,
        [7.51043587e+15],
        [8.10409837e+15],
        [7.41072912e+15]])

In [30]:
del temp
gc.collect()

0

In [183]:
np.power((np.sum(temp,axis=1)/n_s),2)

matrix([[4.94024751e-10],
        [4.94024751e-10],
        [4.94024751e-10],
        ...,
        [4.94024751e-10],
        [4.94024751e-10],
        [1.63860664e-09]])

In [19]:
methods.dynamic_laplacian_sl(x1)

AxisError: axis 1 is out of bounds for array of dimension 1